In [1]:
import pandas as pd

from sklearn import datasets
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import pickle
import joblib

from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor

In [2]:
#scale

def scaling_feat(df, to_scale, not_to_scale, scale=1):
    
    scalable = df[to_scale]
    not_scalable = df[not_to_scale]

    if scale == 2:
        scaler = StandardScaler()
    elif scale ==3:
        scaler = RobustScaler()
    else:
        scaler = MinMaxScaler()

    scaled = pd.DataFrame(scaler.fit_transform(scalable), columns=to_scale)
    
    return not_scalable.join(scaled)

In [3]:
def carat_to_bins(df, carat):
    
    df.loc[df[carat].between(0, .5, 'left'), 'bin'] = 1.0
    df.loc[df[carat].between(.5, 1.0, 'left'), 'bin'] = 1.44
    df.loc[df[carat].between(1.0, 1.5, 'left'), 'bin'] = 2.3
    df.loc[df[carat].between(1.5, 2.0, 'left'), 'bin'] = 3.21
    df.loc[df[carat].between(2.0, 3.0, 'left'), 'bin'] = 4.12
    df.loc[df[carat].between(3.0, 4.0, 'left'), 'bin'] = 5.53
    df.loc[df[carat].between(4.0, 10.0, 'left'), 'bin'] = 8.39
    
    return df

In [4]:
pre_x_train = pd.read_csv('../data/diamonds_train.csv')
pre_x_test = pd.read_csv('../data/diamonds_test.csv')
pre_x_train.drop(columns=['Unnamed: 0'], inplace=True)
pre_x_test.drop(columns=['id'], inplace=True)

In [5]:
#unify with test to scale features

pre_x_train = pd.concat([pre_x_train, pre_x_test])
pre_x_train.reset_index(inplace=True)
pre_x_train.drop(columns=['index'], inplace=True)

In [6]:
#columns bin for carat

pre_x_train = carat_to_bins(pre_x_train, 'carat')

In [7]:
#top size
pre_x_train['xy'] = pre_x_train['x']*pre_x_train['y']

In [ ]:
cut_lbl = {'Premium': 4, 'Very Good': 3, 'Fair' : 1, 'Good': 2, 'Ideal': 5}
color_lbl = {'J': 1, 'H':3, 'G':4, 'D':7, 'F':5, 'E':6, 'I': 2}
clarity_lbl = {'VS2':5, 'VS1':4, 'SI1':2, 'SI2':3, 'IF':8, 'VVS1':6, 'VVS2':7, 'I1':1}

In [20]:
cut2 = [cut_lbl[pre_x_train['cut'][i]] for i in range(len(pre_x_train))]
color2 = [color_lbl[pre_x_train['color'][i]] for i in range(len(pre_x_train))]
clarity2 = [clarity_lbl[pre_x_train['clarity'][i]] for i in range(len(pre_x_train))]

pre_x_train['cut2'] = cut2
pre_x_train['color2'] = color2
pre_x_train['clar2'] = clarity2

pre_x_train.drop(columns=['cut', 'color', 'clarity'], inplace=True)

In [22]:
pre_x_train

,carat,depth,table,x,y,z,city,price,bin,xy,cut2,color2,clar2
0,1.21,62.4,58.0,6.83,6.79,4.25,Dubai,4268.0,2.30,46.3757,4,1,5
1,0.32,63.0,57.0,4.35,4.38,2.75,Kimberly,505.0,1.00,19.0530,3,3,5
2,0.71,65.5,55.0,5.62,5.53,3.65,Las Vegas,2686.0,1.44,31.0786,1,4,4
3,0.41,63.8,56.0,4.68,4.72,3.00,Kimberly,738.0,1.00,22.0896,2,7,2
4,1.02,60.5,59.0,6.55,6.51,3.95,Dubai,4882.0,2.30,42.6405,5,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
53935,0.57,61.9,56.0,5.35,5.32,3.30,Amsterdam,NaN,1.44,28.4620,5,6,2
53936,0.71,62.2,55.0,5.71,5.73,3.56,New York City,NaN,1.44,32.7183,5,2,5
53937,0.70,61.6,55.0,5.75,5.71,3.53,Tel Aviv,NaN,1.44,32.8325,5,5,4
53938,0.70,58.8,57.0,5.85,5.89,3.45,Surat,NaN,1.44,34.4565,3,5,3


In [27]:
#scaling features

feat_to_scale = ['carat', 'depth', 'table', 'x', 'y', 'z', 'bin', 'xy','cut2', 'color2', 'clar2']
others = ['city', 'price']
pre_x_scaled = scaling_feat(pre_x_train, feat_to_scale, others)

In [28]:
#drop test rows
pre_x_scaled = pre_x_scaled[pre_x_scaled['price'].notna()]

In [10]:
#labeling cut, color, clarity

clar_lab_mm = pd.read_csv('../data/scales/clar_lab_mm.csv')
cut_lab_mm = pd.read_csv('../data/scales/cut_lab_mm.csv')
col_lab_mm = pd.read_csv('../data/scales/col_lab_mm.csv')

# clar_lab_rob = pd.read_csv('../data/scales/clar_lab_mm.csv')
# cut_lab_rob = pd.read_csv('../data/scales/cut_lab_mm.csv')
# col_lab_rob = pd.read_csv('../data/scales/col_lab_mm.csv')

clar_lab_mm.drop(columns=['Unnamed: 0'], inplace=True)
cut_lab_mm.drop(columns=['Unnamed: 0'], inplace=True)
col_lab_mm.drop(columns=['Unnamed: 0'], inplace=True)

# clar_lab_rob.drop(columns=['Unnamed: 0'], inplace=True)
# cut_lab_rob.drop(columns=['Unnamed: 0'], inplace=True)
# col_lab_rob.drop(columns=['Unnamed: 0'], inplace=True)

In [11]:
pre_x_scaled2 = pre_x_scaled

cut2 = [cut_lab_mm[cut_lab_mm['cut'] == c]['label'].iloc[0] for c in pre_x_scaled2['cut']]
color2 = [col_lab_mm[col_lab_mm['color'] == c]['label'].iloc[0] for c in pre_x_scaled2['color']]
clarity2 = [clar_lab_mm[clar_lab_mm['clarity'] == c]['label'].iloc[0] for c in pre_x_scaled2['clarity']]

pre_x_scaled2['cut2'] = cut2
pre_x_scaled2['color2'] = color2
pre_x_scaled2['clar2'] = clarity2

In [14]:
cut_lbl = {'Premium': 4, 'Very Good': 3, 'Fair' : 1, 'Good': 2, 'Ideal': 5}
color_lbl = {'J': 1, 'H':3, 'G':4, 'D':7, 'F':5, 'E':6, 'I': 2}
clarity_lbl = {'VS2':5, 'VS1':4, 'SI1':2, 'SI2':3, 'IF':8, 'VVS1':6, 'VVS2':7, 'I1':1}

In [18]:
cut2 = [cut_lbl[pre_x_scaled2['cut'][i]] for i in range(len(pre_x_scaled2))]
color2 = [color_lbl[pre_x_scaled2['color'][i]] for i in range(len(pre_x_scaled2))]
clarity2 = [clarity_lbl[pre_x_scaled2['clarity'][i]] for i in range(len(pre_x_scaled2))]

pre_x_scaled2['cut2'] = cut2
pre_x_scaled2['color2'] = color2
pre_x_scaled2['clar2'] = clarity2

pre_x_scaled2.drop(columns=['cut', 'color', 'clarity'], inplace=True)

In [29]:
pre_x_scaled

,city,price,carat,depth,table,x,y,z,bin,xy,cut2,color2,clar2
0,Dubai,4268.0,0.209979,0.538889,0.288462,0.635940,0.115280,0.133648,0.175913,0.097326,0.75,0.000000,0.571429
1,Kimberly,505.0,0.024948,0.555556,0.269231,0.405028,0.074363,0.086478,0.000000,0.039985,0.50,0.333333,0.571429
2,Las Vegas,2686.0,0.106029,0.625000,0.230769,0.523277,0.093888,0.114780,0.059540,0.065223,0.00,0.500000,0.428571
3,Kimberly,738.0,0.043659,0.577778,0.250000,0.435754,0.080136,0.094340,0.000000,0.046358,0.25,1.000000,0.142857
4,Dubai,4882.0,0.170478,0.486111,0.307692,0.609870,0.110526,0.124214,0.175913,0.089487,1.00,0.500000,0.142857
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,Antwerp,5850.0,0.182952,0.525000,0.211538,0.618250,0.112224,0.128931,0.175913,0.092110,1.00,0.333333,0.571429
40451,Luxembourg,6300.0,0.197505,0.522222,0.230769,0.626629,0.114771,0.131132,0.175913,0.095477,1.00,0.166667,0.142857
40452,Kimberly,1800.0,0.068607,0.511111,0.269231,0.482309,0.088285,0.100314,0.059540,0.056529,1.00,0.666667,0.428571
40453,Amsterdam,2368.0,0.095634,0.494444,0.288462,0.527933,0.095756,0.108176,0.059540,0.067112,0.75,0.666667,0.428571


In [33]:
#droping city column
pre_x_scaled = pre_x_scaled.drop(columns='city')

In [34]:
pre_x_scaled

,price,carat,depth,table,x,y,z,bin,xy,cut2,color2,clar2
0,4268.0,0.209979,0.538889,0.288462,0.635940,0.115280,0.133648,0.175913,0.097326,0.75,0.000000,0.571429
1,505.0,0.024948,0.555556,0.269231,0.405028,0.074363,0.086478,0.000000,0.039985,0.50,0.333333,0.571429
2,2686.0,0.106029,0.625000,0.230769,0.523277,0.093888,0.114780,0.059540,0.065223,0.00,0.500000,0.428571
3,738.0,0.043659,0.577778,0.250000,0.435754,0.080136,0.094340,0.000000,0.046358,0.25,1.000000,0.142857
4,4882.0,0.170478,0.486111,0.307692,0.609870,0.110526,0.124214,0.175913,0.089487,1.00,0.500000,0.142857
...,...,...,...,...,...,...,...,...,...,...,...,...
40450,5850.0,0.182952,0.525000,0.211538,0.618250,0.112224,0.128931,0.175913,0.092110,1.00,0.333333,0.571429
40451,6300.0,0.197505,0.522222,0.230769,0.626629,0.114771,0.131132,0.175913,0.095477,1.00,0.166667,0.142857
40452,1800.0,0.068607,0.511111,0.269231,0.482309,0.088285,0.100314,0.059540,0.056529,1.00,0.666667,0.428571
40453,2368.0,0.095634,0.494444,0.288462,0.527933,0.095756,0.108176,0.059540,0.067112,0.75,0.666667,0.428571


In [35]:
pre_x_scaled.columns

Index(['price', 'carat', 'depth', 'table', 'x', 'y', 'z', 'bin', 'xy', 'cut2',
       'color2', 'clar2'],
      dtype='object')

In [36]:
feat = ['carat', 'bin', 'depth', 'table', 'clar2', 'cut2', 'color2']
# feat = ['price', 'carat', 'depth', 'table', 'x', 'y', 'z', 'bin', 'clar2', 'cut2', 'color2']

target = ['price']

In [43]:
X, y = pre_x_scaled[feat], pre_x_scaled[target]

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
model_rmse = RandomForestRegressor(n_estimators= 512, max_depth= 16,  random_state = 42)


model_rmse.fit(X, y)

C:\Users\magno.MAUDITORS\AppData\Local\Temp\ipykernel_10932\2174967403.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_rmse.fit(X, y)


RandomForestRegressor(max_depth=16, n_estimators=512, random_state=42)

In [45]:
%%time
# XGBoost regression

regressor=xgb.XGBRegressor(eval_metric='rmsle')
regressor.fit(X_train, y_train)
prediction = regressor.predict(X_test)

rmse = mean_squared_error(y_test, prediction)**0.5

print(type(regressor), '\n')
print(rmse, '\n')

<class 'xgboost.sklearn.XGBRegressor'> 

567.5777104959385 

CPU times: total: 2.98 s
Wall time: 396 ms


In [ ]:
%%time

param_grid = {"max_depth":    [6, 8, 16],
              "n_estimators": [512, 700, 900],
              "learning_rate": [0.01, 0.015]}

search = GridSearchCV(regressor, param_grid, cv=5).fit(X_train, y_train)
print("The best hyperparameters are ",search.best_params_)

In [46]:
%%time

regressor=xgb.XGBRegressor(learning_rate = 0.01,
                           n_estimators  = 900,
                           max_depth     = 6,
                           eval_metric='rmsle')

regressor.fit(X_train, y_train)

prediction = regressor.predict(X_test)

rmse = mean_squared_error(y_test, prediction)**0.5

print(type(regressor), '\n')
print(rmse, '\n')

<class 'xgboost.sklearn.XGBRegressor'> 

566.3905611904225 

CPU times: total: 25.5 s
Wall time: 3.72 s


In [ ]:
%%time

#train with whole X

regressor=xgb.XGBRegressor(learning_rate = 0.01,
                           n_estimators  = 900,
                           max_depth     = 6,
                           eval_metric='rmsle')



regressor.fit(X, y)

In [ ]:
## RandomForest regression
model_rmse = RandomForestRegressor(n_estimators= 512, max_depth= 16,  random_state = 42)

model_rmse.fit(X, y)

In [48]:
## lightgbm regression

params = {
    'task': 'train', 
    'boosting': 'gbdt',
    'objective': 'regression',
    'num_leaves': 10,
#     'learnnig_rate': 0.05,
    'metric': {'l2','l1'},
    'verbose': -1
#     'task': 'train', 
#     'boosting': 'gbdt',
#     'objective': 'regression',
#     'num_leaves': 25,
#     'max_depth' :10,
#     'num_iterations': 500,
#     'min_data_in_leaf':25,
#     'metric': {'l2','l1'},
#     'verbose': -1

}


lgb_train = lgb.Dataset(X, y)

model_lightgbm = lgb.train(params,
                 train_set=lgb_train)

In [49]:
%%time

## lightgbm regression

params = {
    'task': 'train', 
    'boosting': 'gbdt',
    'objective': 'regression',
    'num_leaves': 15,
#     'learnnig_rage': 0.05,
    'metric': {'l2','l1'},
    'verbose': -1
#         'task': 'train', 
#     'boosting': 'gbdt',
#     'objective': 'regression',
#     'num_leaves': 25,
#     'max_depth' :10,
#     'num_iterations': 500,
#     'min_data_in_leaf':25,
#     'metric': {'l2','l1'},
#     'verbose': -1
}


lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

model_lightgbm = lgb.train(params,
                 train_set=lgb_train,
                 valid_sets=lgb_eval,
                 early_stopping_rounds=30)


prediction = model_lightgbm.predict(X_test)

rmse = mean_squared_error(y_test, prediction)**0.5

print(rmse, '\n')

[1]	valid_0's l1: 2731.86	valid_0's l2: 1.30601e+07
Training until validation scores don't improve for 30 rounds
[2]	valid_0's l1: 2473.1	valid_0's l2: 1.07771e+07
[3]	valid_0's l1: 2241.19	valid_0's l2: 8.9216e+06
[4]	valid_0's l1: 2032.98	valid_0's l2: 7.41862e+06
[5]	valid_0's l1: 1848.33	valid_0's l2: 6.18944e+06
[6]	valid_0's l1: 1682.71	valid_0's l2: 5.19147e+06
[7]	valid_0's l1: 1536.75	valid_0's l2: 4.38681e+06
[8]	valid_0's l1: 1406.79	valid_0's l2: 3.72855e+06
[9]	valid_0's l1: 1291.7	valid_0's l2: 3.18573e+06
[10]	valid_0's l1: 1188.06	valid_0's l2: 2.74001e+06
[11]	valid_0's l1: 1098.69	valid_0's l2: 2.37437e+06
[12]	valid_0's l1: 1018.55	valid_0's l2: 2.06676e+06
[13]	valid_0's l1: 946.129	valid_0's l2: 1.81348e+06
[14]	valid_0's l1: 883.542	valid_0's l2: 1.60738e+06
[15]	valid_0's l1: 827.703	valid_0's l2: 1.4345e+06
[16]	valid_0's l1: 779.509	valid_0's l2: 1.2941e+06
[17]	valid_0's l1: 736.099	valid_0's l2: 1.17318e+06
[18]	valid_0's l1: 696.65	valid_0's l2: 1.07116e+06


C:\Users\magnof\AppData\Local\miniconda3\envs\sklearn_env\Lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


In [ ]:
%%time
# grid search  hyperparameter tuning
params = {
    'task': 'train', 
    'boosting': 'gbdt',
    'objective': 'regression',
    'num_leaves': 25,
    'max_depth' :10,
    'num_iterations': 500,
    'min_data_in_leaf':25,
    'metric': {'l2','l1'},
    'verbose': -1
}


lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

model_lightgbm = lgb.train(params,
                 train_set=lgb_train,
                 valid_sets=lgb_eval,
                 early_stopping_rounds=30)


prediction = model_lightgbm.predict(X_test)

rmse = mean_squared_error(y_test, prediction)**0.5

print(rmse, '\n')

## Preparing TEST DF

In [20]:
#preparing test df

to_predict = pre_x_test = pd.read_csv('../data/diamonds_test.csv')
to_predict.drop(columns=['id'], inplace=True)


#carat to bins
to_predict = carat_to_bins(to_predict, 'carat')

#scaling columns
feat_to_scale = ['carat', 'depth', 'table', 'x', 'y', 'z', 'bin']
others = ['cut', 'color', 'clarity', 'city']
to_predict_s = scaling_feat(to_predict, feat_to_scale, others)

In [21]:
#labeling cut, color, clarity

cut2 = [cut_lab_mm[cut_lab_mm['cut'] == c]['label'].iloc[0] for c in to_predict_s['cut']]
color2 = [col_lab_mm[col_lab_mm['color'] == c]['label'].iloc[0] for c in to_predict_s['color']]
clarity2 = [clar_lab_mm[clar_lab_mm['clarity'] == c]['label'].iloc[0] for c in to_predict_s['clarity']]

to_predict_s['cut2'] = cut2
to_predict_s['color2'] = color2
to_predict_s['clar2'] = clarity2

to_predict_s.drop(columns=['cut', 'color', 'clarity'], inplace=True)

In [22]:
#drop city column
to_predict_s.drop(columns=['city'], inplace=True)

In [23]:
to_predict_s.columns

Index(['carat', 'depth', 'table', 'x', 'y', 'z', 'bin', 'cut2', 'color2',
       'clar2'],
      dtype='object')

In [24]:
to_predict_X = to_predict_s[feat]

In [26]:
#predictions
# final_prediction = regressor.predict(to_predict_X)

# #lightgbm predictions
# light_pred = model_lightgbm.predict(to_predict_X)

# #catboost predictions
# cat_pred = model_cat.predict(to_predict_X)

#RandonForest predictions
rforest_pred = model_rmse.predict(to_predict_X)

In [27]:
prices = pd.DataFrame(rforest_pred, columns=['Price'])
prices.reset_index(inplace=True)
prices.rename(columns={"index": "id"}, inplace=True)
prices = prices.set_index('id')
prices.to_csv('../data/prices_015.csv')

prices

,Price
id,
0,2955.979272
1,5377.005074
2,10408.099352
3,4168.651188
4,1656.621385
...,...
13480,1481.577051
13481,2882.972504
13482,3096.961191


In [ ]:
def classify_diamond_shape(x, y, z, table, depth):
    # Calculate ratios
    length_to_width_ratio = x / y
    length_to_depth_ratio = x / z

    # Define shape classification ranges
    shape_ranges = {
        'Round Brilliant': {
            'length_to_width_ratio': (0.95, 1.05),
            'length_to_depth_ratio': (1.35, 1.65),
            'table_percentage': (52, 62),
            'depth_percentage': (59, 62)
        },
        'Princess': {
            'length_to_width_ratio': (0.90, 1.10),
            'length_to_depth_ratio': (0.90, 1.10),
            'table_percentage': (60, 80),
            'depth_percentage': (58, 77)
        },
        'Emerald': {
            'length_to_width_ratio': (1.20, 1.60),
            'length_to_depth_ratio': (1.30, 1.60),
            'table_percentage': (60, 69),
            'depth_percentage': (60, 75)
        },
        'Asscher': {
            'length_to_width_ratio': (1.00, 1.05),
            'length_to_depth_ratio': (1.00, 1.05),
            'table_percentage': (60, 68),
            'depth_percentage': (60, 75)
        },
        'Radiant': {
            'length_to_width_ratio': (1.00, 1.30),
            'length_to_depth_ratio': (1.00, 1.30),
            'table_percentage': (60, 69),
            'depth_percentage': (60, 75)
        },
        'Pear': {
            'length_to_width_ratio': (1.40, 2.00),
            'length_to_depth_ratio': (1.30, 1.70),
            'table_percentage': (53, 63),
            'depth_percentage': (58, 66)
        },
        'Marquise': {
            'length_to_width_ratio': (1.60, 2.10),
            'length_to_depth_ratio': (1.30, 2.20),
            'table_percentage': (53, 63),
            'depth_percentage': (58, 66)
        },
        'Oval': {
            'length_to_width_ratio': (1.20, 1.70),
            'length_to_depth_ratio': (1.30, 1.70),
            'table_percentage': (53, 63),
            'depth_percentage': (58, 66)
        },
        'Heart': {
            'length_to_width_ratio': (0.90, 1.10),
            'length_to_depth_ratio': (0.90, 1.10),
            'table_percentage': (53, 63),
            'depth_percentage': (58, 66)
        },
        'Cushion': {
            'length_to_width_ratio': (1.00, 1.10),
            'length_to_depth_ratio': (1.00, 1.10),
            'table_percentage': (53, 63),
            'depth_percentage': (58, 66)
        }
    }

    # Classify the diamond shape
    for shape, ranges in shape_ranges.items():
        if (ranges['length_to_width_ratio'][0] <= length_to_width_ratio <= ranges['length_to_width_ratio'][1] and
                ranges['length_to_depth_ratio'][0] <= length_to_depth_ratio <= ranges['length_to_depth_ratio'][1] and
                ranges['table_percentage'][0] <= table <= ranges['table_percentage'][1] and
                ranges['depth_percentage'][0] <= depth <= ranges['depth_percentage'][1]):
            return shape

    return 'Unknown'  # If no shape match is found

# Example usage
x = 6.1
y = 6.1
z = 3.7
table = 58
depth = 61

diamond_shape = classify_diamond_shape(x, y, z, table, depth)
print("The diamond shape is:", diamond_shape)


In [ ]:
len(pre_x_train)

In [ ]:
shapes = []

for i in range(len(pre_x_train)):
    x = pre_x_train['x'].iloc[i]
    y = pre_x_train['y'].iloc[i]
    z = pre_x_train['z'].iloc[i]
    t = pre_x_train['table'].iloc[i]
    d = pre_x_train['depth'].iloc[i]

    shapes.append(classify_diamond_shape(x, y, z, t, d))
    
shapes

In [ ]:
pre_x_train

In [ ]:
xy = 6.83/6.79
xz = 6.83/4.25
# xy
xz